In [1]:
import numpy as np
from Druckrohrleitung_class_file import Druckrohrleitung_class
import matplotlib.pyplot as plt

#importing pressure conversion function
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion

In [2]:
%matplotlib qt5
#define constants

g       = 9.81              # gravitational acceleration [m/s²]
rho     = 1000              # density of water [kg/m³]

L       = 1000              # length of pipeline [m]
D       = 1                 # pipe diameter [m]
Q0      = 2                 # initial flow in whole pipe [m³/s]
h_res   = 20                # water level in upstream reservoir [m]
n       = 10                # number of pipe segments in discretization
nt      = 100               # number of time steps after initial conditions
f_D     = 0.01              # Darcy friction factor
c       = 400               # propagation velocity of the pressure wave [m/s]
h_pipe  = 200              # hydraulic head without reservoir [m] 
alpha   = np.arcsin(h_pipe/L)  # Höhenwinkel der Druckrohrleitung 


# preparing the discretization and initial conditions
initial_influx  = 2.                                    # m³/s
initial_level   = 10.                                   # m
dx              = L/n                                   # length of each pipe segment
dt              = dx/c                                  # timestep according to method of characterisitics
nn              = n+1                                   # number of nodes
pl_vec          = np.arange(0,nn*dx,dx)                 # pl = pipe-length. position of the nodes on the pipeline
t_vec           = np.arange(0,nt*dt,dt)                 # time vector
h_vec           = np.arange(0,h_pipe+h_pipe/n,h_pipe/n) # hydraulic head of pipeline at each node


In [3]:
pipe = Druckrohrleitung_class(L,D,n,alpha,f_D)
pipe.set_pressure_propagation_velocity(c)
pipe.set_number_of_timesteps(nt)
pipe.set_steady_state(initial_influx,initial_level,pl_vec,h_vec)

In [4]:
# initialization for timeloop

# prepare the vectors in which the pressure and velocity distribution in the pipeline from the previous timestep are stored
v_old = pipe.get_current_velocity_distribution()
p_old = pipe.get_current_pressure_distribution()

# prepare the vectors in which the temporal evolution of the boundary conditions are stored
    # keep in mind, that the velocity at the turbine and the pressure at the reservoir are set manually and
        # through the time evolution of the reservoir respectively  
    # the pressure at the turbine and the velocity at the reservoir are calculated from the method of characteristics
v_boundary_res  = np.zeros_like(t_vec)
v_boundary_tur  = np.zeros_like(t_vec)
p_boundary_res  = np.zeros_like(t_vec)
p_boundary_tur  = np.zeros_like(t_vec)

# set the boundary conditions for the first timestep
v_boundary_res[0]   = v_old[0]
v_boundary_tur[0]   = v_old[-1] 
p_boundary_res[0]   = p_old[0]
p_boundary_tur[0]   = p_old[-1]


In [5]:
fig2,axs2 = plt.subplots(2,1)
axs2[0].set_title('Pressure distribution in pipeline')
axs2[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs2[0].set_ylabel(r'$p$ [mWS]')
lo_00, = axs2[0].plot(pl_vec,pressure_conversion(p_old,'Pa','mWS'),marker='.')
axs2[0].set_ylim([0.9*np.min(pressure_conversion(p_old,'Pa','mWS')),1.1*np.max(pressure_conversion(p_old,'Pa','mWS'))])

axs2[1].set_title('Velocity distribution in pipeline')
axs2[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs2[1].set_ylabel(r'$p$ [mWS]')
lo_01, = axs2[1].plot(pl_vec,v_old,marker='.')
axs2[1].set_ylim([0.9*np.min(v_old),1.1*np.max(v_boundary_res)])

fig2.tight_layout()
plt.pause(5)


for it in range(1,pipe.nt):
    pipe.set_boundary_conditions_next_timestep(p_boundary_res[0],v_boundary_tur[0])
    pipe.timestep_characteristic_method()
    lo_00.set_ydata(pressure_conversion(pipe.get_current_pressure_distribution(),'Pa','mWS'))
    lo_01.set_ydata(pipe.get_current_velocity_distribution())

    v_boundary_res[it] = pipe.get_current_velocity_distribution()[0]
    v_boundary_tur[it] = pipe.get_current_velocity_distribution()[-1]
    p_boundary_res[it] = pipe.get_current_pressure_distribution()[0]
    p_boundary_tur[it] = pipe.get_current_pressure_distribution()[-1]


    
    fig2.suptitle(str(it))
    fig2.canvas.draw()
    fig2.tight_layout()
    plt.pause(0.2)


In [6]:
fig3,axs3 = plt.subplots(2,2)
axs3[0,0].set_title('Pressure Reservoir')
axs3[0,0].plot(t_vec,pressure_conversion(p_boundary_res,'Pa','mWS'))
axs3[0,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs3[0,0].set_ylabel(r'$p$ [mWS]')
axs3[0,0].set_ylim([0.9*np.min(pressure_conversion(p_boundary_res,'Pa','mWS')),1.1*np.max(pressure_conversion(p_boundary_res,'Pa','mWS'))])

axs3[0,1].set_title('Velocity Reservoir')
axs3[0,1].plot(t_vec,v_boundary_res)
axs3[0,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs3[0,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs3[0,1].set_ylim([0.9*np.min(v_boundary_res),1.1*np.max(v_boundary_res)])

axs3[1,0].set_title('Pressure Turbine')
axs3[1,0].plot(t_vec,pressure_conversion(p_boundary_tur,'Pa','mWS'))
axs3[1,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs3[1,0].set_ylabel(r'$p$ [mWS]')
axs3[1,0].set_ylim([0.9*np.min(pressure_conversion(p_boundary_tur,'Pa','mWS')),1.1*np.max(pressure_conversion(p_boundary_tur,'Pa','mWS'))])

axs3[1,1].set_title('Velocity Turbine')
axs3[1,1].plot(t_vec,v_boundary_tur)
axs3[1,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs3[1,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs3[1,1].set_ylim([0.9*np.min(v_boundary_tur),1.1*np.max(v_boundary_tur)])

fig3.tight_layout()
plt.show()